In [2]:
from torch import nn

from multi_modal_edge_ai.adl_inference.data_access.parser import parse_file
from multi_modal_edge_ai.adl_inference.ml_models.cnn_model import CNNModel
from multi_modal_edge_ai.adl_inference.preprocessing.encoder import Encoder
from multi_modal_edge_ai.adl_inference.validating.validate import split_and_validate


(sdf, adf) = parse_file(r'../public_dataset/OrdonezA_Sensors.csv', r'../public_dataset/OrdonezA_ADLs.csv')

adls = adf['Activity'].unique().tolist()
adl_encoder = Encoder(adls)

# If you want to replace the existing 'Activity' column with encoded values
adf['Activity'] = adf['Activity'].apply(adl_encoder.encode_label)

# Assuming df is your DataFrame

sensors = sdf['Sensor'].unique().tolist()

model = CNNModel(6, 2, 400, sensors, len(adls), nn.ReLU(), nn.LogSoftmax(dim=-1))

In [3]:
result = split_and_validate(data=sdf, ground_truth=adf, model=model, labels=adls, label_encoder = adl_encoder, window_length_seconds=400, window_slide_seconds=200)



Training started....
Epoch 1/1, Loss: 1306.5473027204837


Training completed.


In [4]:
import pandas as pd

print('\n')
print('Accuracy =' + str(result[0]))
cm = result[1]
cm = pd.DataFrame(cm, columns=adls)
pd.set_option("display.max_columns", 10)
print(cm)




Accuracy =0.9182732606873428
   Sleeping  Idle  Bathroom_Usage  Meal_Preparation  Relax  Outside
0       950     0               0                 0      0        0
1         9    31               0                 0     34       61
2         2     3               0                 0      0       25
3        23     6               0                 0      0       18
4         0     6               0                 0   1030        4
5         0     2               0                 0      2      180
